# Primary Outcome:  

Proportion of patients who contact NHS111 and are triaged to an urgent dental disposition who see a dentist face-to-face within 7 days of the index call 

  

# Secondary outcomes:  

Proportion of patients who contact NHS111 and are triaged to an urgent dental disposition who see a dentist face-to-face within 24 hours (NHS commissioning guidance) 

Proportion of patients who contact NHS111 and are triaged to an urgent dental disposition who: 

See a dentist face-to-face within the disposition timeframe 

Are admitted or discharged from ED 

Have contact with a non-dental primary care service 

Number of 111 contacts within a year stratified by IMD quintile/decile 

Mean/median distance from home LSOA to first dental face-to-face appointment stratified by quintile 

# Date range
All index 111 calls occuring between 1st October 2022 and 31st October 2024

In [ ]:
library(DBI)
library(dbplyr)
library(bigrquery)
suppressPackageStartupMessages(library(tidyverse))
library(lubridate)
library(readxl)

In [ ]:
options(tibble.width = Inf, width = 300)

In [ ]:
# The page size was adjusted as bigquery was throwing errors
con <- DBI::dbConnect(
    bigquery(), 
    project = "yhcr-prd-bradfor-bia-core", 
    # bigint = "integer64", 
    page_size = 15000,
    dataset = 'CB_2617'
)


In [ ]:
dbListTables(con)

In [ ]:
iuc_tbl <- tbl(con, "tbl_YAS_IUC")

In [ ]:
iuc_tbl  %>% colnames()

In [ ]:
iuc_tbl %>% count()

In [ ]:
iuc_tbl %>% filter(!is.na(person_id)) %>% count()

# Date distribution of calls

In [ ]:
iuc_tbl %>% summarise(
    min_date = min(tbl_YAS_IUC_start_date, na.rm = T),
    max_date = max(tbl_YAS_IUC_start_date, na.rm = T)
)

# Number of index calls

In [ ]:
sql <- "
    WITH ordered_calls AS (
      SELECT
        person_id,
        tbl_YAS_IUC_start_date,
        LAG(tbl_YAS_IUC_start_date) OVER (PARTITION BY person_id ORDER BY tbl_YAS_IUC_start_date) AS previous_call
      FROM `tbl_YAS_IUC`
     WHERE person_id IS NOT NULL
    ),
    session_marks AS (
      SELECT
        person_id,
        tbl_YAS_IUC_start_date,
        CASE
          WHEN previous_call IS NULL THEN 1
          WHEN DATE_DIFF(tbl_YAS_IUC_start_date, previous_call, DAY) > 7 THEN 1
          ELSE 0
        END AS new_session
      FROM ordered_calls
    ),
    sessions AS (
      SELECT
        person_id,
        tbl_YAS_IUC_start_date,
        SUM(new_session) OVER (PARTITION BY person_id ORDER BY tbl_YAS_IUC_start_date) AS session_id
      FROM session_marks
    ),
    session_groups AS (
      SELECT
        person_id,
        session_id,
        MIN(tbl_YAS_IUC_start_date) AS index_call,
        COUNT(*) AS call_count
      FROM sessions
      GROUP BY person_id, session_id
    )
    SELECT * FROM session_groups
    ORDER BY person_id, index_call;
"

In [ ]:
result <- dbGetQuery(con, sql)

In [ ]:
result %>% arrange(desc(call_count)) %>% head(n=20)

# Save IUC lookup tables

In [ ]:
dx_code_df <- read_xlsx('../data/IUC-ADC-REVISED-Dx-code-mapping-October-2023.xlsx', skip = 1) %>% transmute(
    dx_code = tolower(`Disposition Code`),
    dx_desc = `Disposition Text` 
)

In [ ]:
dx_code_df %>% glimpse()

In [ ]:
# dbWriteTable(con, "iuc_dx_lookup", dx_code_df, overwrite = T) 

In [ ]:
iuc_dx_lookup_tbl <- tbl(con, "iuc_dx_lookup")

In [ ]:
iuc_dx_lookup_tbl %>% glimpse()

# Index calls redux

In [ ]:
iuc_tbl %>% 
    mutate(
        final_dx_code = tolower(final_dx_code)
    ) %>% 
    # left_join(iuc_dx_lookup_tbl, by=c("final_dx_code"="dx_code")) %>%
    filter(person_id == "PERSON_ID") %>% #head()
    arrange(tbl_YAS_IUC_start_date) %>% 
    select(
        tbl_YAS_IUC_start_date, final_dx_description, referral_service
    ) %>% print(n=50)

In [ ]:
# iuc_tbl %>% 
#     mutate(
#         final_dx_code = tolower(final_dx_code)
#     ) %>% show_query()

# Check Disposition code

In [ ]:
# iuc_dx_lookup_tbl <- tbl(con, "iuc_dx_lookup")

In [ ]:
iuc_tbl %>% 
    mutate(
        final_dx_code = tolower(final_dx_code)
    ) %>%
    # left_join(iuc_dx_lookup_tbl, by=c("final_dx_code"="dx_code")) %>%
    count(final_dx_description, sort = T)

# Check distribution of calls

In [ ]:
monthly_counts <- result %>%
  mutate(month = floor_date(index_call, "month")) %>%
  group_by(month) %>%
  summarise(index_call_count = n()) %>%
  arrange(month)

In [ ]:
monthly_counts %>% glimpse()

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8) 

In [ ]:
ggplot(monthly_counts, aes(x = month, y = index_call_count)) +
  geom_col() +
  labs(
    title = "Monthly Count of Index Calls",
    x = "Month",
    y = "Number of Index Calls"
  ) +
  theme_minimal()

In [ ]:
iuc_tbl %>%
    mutate(month = sql("DATE_TRUNC(tbl_YAS_IUC_start_date, MONTH)")) %>%
      count(month) %>%
      arrange(month) %>%
ggplot(aes(x = month, y = n)) +
geom_col()
    

In [ ]:
total_iuc_df <- read_csv('../data/IUC-total-count.csv')

In [ ]:
#dbWriteTable(con, "iuc_total_count", total_iuc_df, overwrite = T) 

In [ ]:
iuc_total_count_tbl <- tbl(con, "iuc_total_count")

In [ ]:
iuc_total_count_tbl

In [ ]:
iuc_count_df <- iuc_tbl %>%
    mutate(month = sql("CAST(DATE_TRUNC(tbl_YAS_IUC_start_date, MONTH) AS DATE)")) %>% 
    left_join(iuc_total_count_tbl, by=c("month"="Date")) %>% 
    select(month, Count) %>%
    group_by(month) %>%
    summarise(
        dental = n(),
        total = max(Count)
    ) %>%
    arrange(month) %>%
    collect()

In [ ]:
iuc_count_df %>% 
    pivot_longer(-month) %>%
    ggplot(aes(x = month, y = value, fill = name)) +
    geom_col() +
    facet_wrap(~name, scales="free_y")

In [ ]:
dental_tbl <- tbl(con, "tbl_Dental_Data_NHBSA")

In [ ]:
dental_tbl %>% colnames()

In [ ]:
dental_tbl %>% summarise(
    min_date = min(tbl_Dental_Data_NHBSA_start_date, na.rm = T),
    max_date = max(tbl_Dental_Data_NHBSA_start_date, na.rm = T)
    )

In [ ]:
monthly_dental_counts <- dental_tbl %>%
  filter(between(tbl_Dental_Data_NHBSA_start_date, '2022-10-01', '2023-10-31')) %>%
  mutate(month = sql("CAST(DATE_TRUNC(tbl_Dental_Data_NHBSA_start_date, MONTH) AS DATE)")) %>% 
  count(month) %>% rename(dental_n = n) %>% collect()

In [ ]:
monthly_dental_counts %>% print(n=20)

In [ ]:
iuc_count_df <- iuc_tbl %>%
    mutate(month = sql("CAST(DATE_TRUNC(tbl_YAS_IUC_start_date, MONTH) AS DATE)")) %>% 
    left_join(iuc_total_count_tbl, by=c("month"="Date")) %>% 
    select(month, Count) %>%
    group_by(month) %>%
    summarise(
        dental = n(),
        total = max(Count)
    ) %>%
    arrange(month) %>%
    collect()

In [ ]:
iuc_dental_count_df %>% glimpse()

In [ ]:
dental_iuc_df <- iuc_count_df %>% left_join(monthly_dental_counts)

In [ ]:
dental_iuc_df %>% glimpse()

In [ ]:
dental_iuc_df %>%
    select(-total) %>%
    pivot_longer(-month) %>%
    ggplot(aes(x = month, y = value, fill = name)) +
    geom_col()

In [ ]:
dental_tbl %>% count()

In [ ]:
dental_tbl %>% count(TREATMENT_CHARGE_BAND)

In [ ]:
# Combine IUC and dental
NOTE: a bit hacky....might have to refactor

In [ ]:
# Looks to be people from BD and LS postcode
# Filtering IUC data on dental data, although this will not then provide everyone
# possibly, so might have to sort something else for the denominator

dental_sql <- "WITH ordered_calls AS (
  SELECT
    iuc.person_id,
    iuc.final_dx_description,
    iuc.tbl_YAS_IUC_start_date,
    LAG(iuc.tbl_YAS_IUC_start_date) OVER (PARTITION BY iuc.person_id ORDER BY iuc.tbl_YAS_IUC_start_date) AS previous_call
  FROM `tbl_YAS_IUC` iuc
  WHERE iuc.person_id IN (
    SELECT DISTINCT person_id FROM `tbl_Dental_Data_NHBSA`
  )
),

session_marks AS (
  SELECT
    person_id,
    tbl_YAS_IUC_start_date,
    CASE
      WHEN previous_call IS NULL THEN 1
      WHEN DATE_DIFF(tbl_YAS_IUC_start_date, previous_call, DAY) > 7 THEN 1
      ELSE 0
    END AS new_session
  FROM ordered_calls
),

sessions AS (
  SELECT
    person_id,
    tbl_YAS_IUC_start_date,
    SUM(new_session) OVER (PARTITION BY person_id ORDER BY tbl_YAS_IUC_start_date) AS session_id
  FROM session_marks
),

session_groups AS (
  SELECT
    person_id,
    session_id,
    MIN(tbl_YAS_IUC_start_date) AS index_call,
    MAX(tbl_YAS_IUC_start_date) AS last_call_in_session
  FROM sessions
  GROUP BY person_id, session_id
),

session_with_dx AS (
  SELECT
    sg.*,
    iy.final_dx_description
  FROM session_groups sg
  LEFT JOIN `tbl_YAS_IUC` iy
    ON sg.person_id = iy.person_id
    AND sg.index_call = iy.tbl_YAS_IUC_start_date
),

dental_matches AS (
  SELECT
    s.person_id,
    s.session_id,
    s.index_call,
    s.last_call_in_session,
    s.final_dx_description,
    d.tbl_Dental_Data_NHBSA_start_date AS dental_date,
    d.TREATMENT_CHARGE_BAND
  FROM session_with_dx s
  LEFT JOIN `tbl_Dental_Data_NHBSA` d
    ON s.person_id = d.person_id
    AND d.tbl_Dental_Data_NHBSA_start_date BETWEEN s.index_call AND DATE_ADD(s.index_call, INTERVAL 7 DAY)
)

SELECT * 
FROM dental_matches
ORDER BY person_id, index_call, dental_date;

"


In [ ]:
dental_result <- dbGetQuery(con, dental_sql)

In [ ]:
dental_result %>% arrange(person_id) %>% count(TREATMENT_CHARGE_BAND)

In [ ]:
dental_result %>% arrange(person_id) %>% count(!is.na(dental_date)) # 11,914

In [ ]:
dental_result %>% arrange(person_id) %>% filter(!is.na(dental_date)) %>% count(final_dx_description) %>% arrange(desc(n))

In [ ]:
dental_tbl %>% count(PARTIAL_POSTCODE)

In [ ]:
2500+1283+435+141+99+69

In [ ]:
# Data looks to be BD and LS postcodes only. Will need to filter IUC data on these
# Probably further filtering required v 

In [ ]:
# Data dictionary of sorts for dental

In [ ]:
project <- "yhcr-prd-bradfor-bia-core"
dataset <- "CB_2617"
table_name <- "tbl_Dental_Data_NHBSA"

# Step 1: Get column metadata
columns <- dbGetQuery(con, glue::glue("
  SELECT column_name, data_type 
  FROM `{project}.{dataset}.INFORMATION_SCHEMA.COLUMNS`
  WHERE table_name = '{table_name}'
"))


In [ ]:
# Step 2: For each column, compute profiling stats
profile_column <- function(...) {
  args <- list(...)
  colname <- args$column_name
  dtype <- args$data_type

  base_query <- glue::glue("
    SELECT
      COUNT(*) AS total_rows,
      COUNTIF({colname} IS NULL) / COUNT(*) AS null_pct
  ")

  details <- dplyr::case_when(
    dtype %in% c("STRING", "BOOL") ~ glue::glue(", COUNT(DISTINCT {colname}) AS n_unique,
      ARRAY_AGG(DISTINCT {colname} IGNORE NULLS) AS sample_values"),
    dtype %in% c("INT64", "NUMERIC", "FLOAT64") ~ glue::glue(", MIN({colname}) AS min_val,
      MAX({colname}) AS max_val"),
    dtype == "DATE" ~ glue::glue(", MIN({colname}) AS min_date,
      MAX({colname}) AS max_date"),
    TRUE ~ ""
  )

  full_query <- glue::glue("{base_query} {details} FROM `{project}.{dataset}.{table_name}`")

  tryCatch({
    DBI::dbGetQuery(con, full_query) %>%
      mutate(column_name = colname, data_type = dtype)
  }, error = function(e) {
    tibble(column_name = colname, data_type = dtype, error = e$message)
  })
}




In [ ]:
# Step 3: Map profiling over all columns
data_dict <- pmap_dfr(columns, profile_column)

# Step 4: Reorder and view
data_dict <- data_dict %>% 
  select(column_name, data_type, total_rows, null_pct, everything()) %>%
  arrange(desc(null_pct))

In [ ]:
data_dict %>% glimpse()

In [ ]:
data_dict_clean <- data_dict %>%
  mutate(
    sample_values = purrr::map_chr(sample_values, ~ if (is.null(.x)) NA_character_ else paste(.x, collapse = "; "))
  )

In [ ]:
data_dict_clean

In [ ]:
data_dict_clean %>% write_csv('../data/dental_data_dict.csv')